In [39]:
import pandas as pd
import getEPH
import statsmodels.formula.api as slm
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [40]:
#getEPH.getEPHdbf('t310')

In [41]:
data = pd.read_csv('data/cleanDataHouseholdt310.csv')
data.head()

,CODUSU,NRO_HOGAR,REGION,PONDERA,HomeType,HomeTypeesp,RoomsNumber,FloorMaterial,FloorMaterialesp,RoofMaterial,...,HouseMembers,Memberless10,Membermore10,TotalHouseHoldIncome,DomesticService1,DomesticService2,DomesticService3,DomesticService4,DomesticService5,DomesticService6
0,302468,1,1,1287,2,NaN,2,1,NaN,9,...,2,0,2,4000.0,1,0,2,0,0,0
1,307861,1,1,1674,2,NaN,2,1,NaN,1,...,4,1,3,5800.0,1,2,98,0,0,0
2,308762,1,1,1522,2,NaN,4,1,NaN,9,...,1,0,1,3200.0,1,0,98,0,0,0
3,308278,1,1,1320,2,NaN,3,1,NaN,9,...,2,0,2,10000.0,2,0,96,0,0,0
4,311937,1,1,1281,2,NaN,4,1,NaN,1,...,4,0,4,11000.0,2,4,96,0,0,0


In [42]:
data = data.query('REGION == 1')
data = data.dropna(axis = 1)
data["id"] = data.CODUSU.map(str) + data.NRO_HOGAR.map(str)
data = data.drop(['CODUSU','NRO_HOGAR', 'DomesticService1','DomesticService2',
                  'DomesticService3','DomesticService4','DomesticService5','DomesticService6'],axis = 1)
def remove9(df,variables):
    for var in variables:
        df[var].replace(to_replace=[9], value=[np.nan] , inplace=True, axis=None)

def remove0(df,variables):
    for var in variables:
        df[var].replace(to_replace=[0], value=[np.nan] , inplace=True, axis=None)

def remove99(df,variables):
    for var in variables:
        df[var].replace(to_replace=[99], value=[np.nan] , inplace=True, axis=None)

        
remove9(df = data, variables = ['FloorMaterial','RoofMaterial','RoofCoat','Water','WaterType','Toilet','ToiletLocation',
                               'ToiletType','Sewer','DumpSites','Flooding','EmergencyLoc','CookingCombustible',
                               'BathroomUse'])
remove0(df = data, variables = ['FloorMaterial','RoofMaterial','RoofCoat','Water','WaterType','Toilet','ToiletLocation',
                               'ToiletType','Sewer','DumpSites','Flooding','EmergencyLoc','Ownership','CookingCombustible',
                               'BathroomUse'])
remove99(df = data, variables = ['Ownership'])

In [43]:
cols = data.columns.tolist()
cols = cols[-1:] + cols[:-1]
data = data[cols]
data.TotalHouseHoldIncome = data.TotalHouseHoldIncome / 1000

In [44]:
df1 = data.iloc[:,3:]

lm = slm.ols(formula = 'TotalHouseHoldIncome ~ ' + ' + '.join(df1.columns[:-1]), data = df1).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     TotalHouseHoldIncome   R-squared:                       0.261
Model:                              OLS   Adj. R-squared:                  0.252
Method:                   Least Squares   F-statistic:                     29.77
Date:                  Mon, 21 Nov 2016   Prob (F-statistic):          7.35e-124
Time:                          17:02:05   Log-Likelihood:                -5887.0
No. Observations:                  2224   AIC:                         1.183e+04
Df Residuals:                      2197   BIC:                         1.198e+04
Df Model:                            26                                         
Covariance Type:              nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept              1.5898      0.894      1.777      0.076        -0.164     3.344
HomeType               0.3472      0.161      2.162      0.031         0.032     0.662
RoomsNumber            0.0095      0.363      0.026      0.979        -0.702     0.721
FloorMaterial         -0.4694      0.244     -1.923      0.055        -0.948     0.009
RoofMaterial          -0.0723      0.066     -1.092      0.275        -0.202     0.057
RoofCoat              -0.0825      0.233     -0.354      0.723        -0.539     0.374
Water                  0.2319      0.366      0.634      0.526        -0.485     0.949
WaterType             -0.3012      0.197     -1.529      0.126        -0.688     0.085
Toilet                 1.5898      0.894      1.777      0.076        -0.164     3.344
ToiletLocation        -0.2641      0.405     -0.652      0.514        -1.058     0.530
ToiletType            -0.2386      0.311     -0.768      0.442        -0.848     0.370
Sewer                 -0.3672      0.134     -2.750      0.006        -0.629    -0.105
DumpSites              0.1584      0.269      0.588      0.557        -0.370     0.687
Flooding               0.5922      0.210      2.816      0.005         0.180     1.005
EmergencyLoc           0.3665      0.706      0.519      0.604        -1.018     1.751
UsableTotalRooms       0.6396      0.368      1.736      0.083        -0.083     1.362
SleepingRooms          0.2025      0.149      1.359      0.174        -0.090     0.495
OfficeRooms            0.8646      0.260      3.324      0.001         0.355     1.375
OnlyWork               0.4155      0.154      2.690      0.007         0.113     0.718
Kitchen                1.0693      0.292      3.665      0.000         0.497     1.641
Sink                  -0.4334      0.160     -2.717      0.007        -0.746    -0.121
Garage                -0.3789      0.170     -2.229      0.026        -0.712    -0.046
Ownership             -0.0609      0.048     -1.265      0.206        -0.155     0.033
CookingCombustible    -0.7769      0.204     -3.803      0.000        -1.178    -0.376
BathroomUse            0.0132      0.266      0.050      0.960        -0.509     0.535
Working               -2.5302      0.215    -11.753      0.000        -2.952    -2.108
HouseMembers           0.0685      0.040      1.717      0.086        -0.010     0.147
Memberless10          -0.2836      0.069     -4.103      0.000        -0.419    -0.148
Membermore10           0.3521      0.058      6.088      0.000         0.239     0.466
==============================================================================
Omnibus:                     1986.310   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           150117.549
Skew:                           3.864   

In [45]:
PV = []
feats = df1.columns[:-1]
for feat in feats:
    if lm.pvalues[str(feat)] < 0.05:
        PV.append(feat)
df1 = pd.concat([df1['TotalHouseHoldIncome'],df1[PV]],axis = 1)
df1.head()

,TotalHouseHoldIncome,HomeType,Sewer,Flooding,OfficeRooms,OnlyWork,Kitchen,Sink,Garage,CookingCombustible,Working,Memberless10,Membermore10
0,4.0,2,1.0,2.0,2,0,1,2,2,1.0,2,0,2
1,5.8,2,1.0,2.0,2,0,1,2,2,1.0,1,1,3
2,3.2,2,1.0,1.0,2,0,1,1,2,1.0,2,0,1
3,10.0,2,1.0,2.0,2,0,1,2,2,4.0,1,0,2
4,11.0,2,1.0,1.0,2,0,1,2,2,1.0,1,0,4


In [46]:
lm2 = slm.ols(formula = 'TotalHouseHoldIncome ~ ' + ' + '.join(df1.columns[1:]), data = df1).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     TotalHouseHoldIncome   R-squared:                       0.217
Model:                              OLS   Adj. R-squared:                  0.214
Method:                   Least Squares   F-statistic:                     61.31
Date:                  Mon, 21 Nov 2016   Prob (F-statistic):          1.80e-131
Time:                          17:02:14   Log-Likelihood:                -7196.7
No. Observations:                  2662   AIC:                         1.442e+04
Df Residuals:                      2649   BIC:                         1.450e+04
Df Model:                            12                                         
Covariance Type:              nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept              7.1426      0.840      8.506      0.000         5.496     8.789
HomeType               0.4410      0.145      3.050      0.002         0.157     0.725
Sewer                 -0.7449      0.123     -6.045      0.000        -0.987    -0.503
Flooding               0.8700      0.207      4.207      0.000         0.464     1.275
OfficeRooms            0.4045      0.256      1.583      0.114        -0.097     0.906
OnlyWork               0.4836      0.148      3.263      0.001         0.193     0.774
Kitchen                0.6691      0.286      2.342      0.019         0.109     1.229
Sink                  -0.7970      0.149     -5.339      0.000        -1.090    -0.504
Garage                -0.4947      0.169     -2.924      0.003        -0.826    -0.163
CookingCombustible    -1.2507      0.180     -6.967      0.000        -1.603    -0.899
Working               -2.6103      0.199    -13.144      0.000        -3.000    -2.221
Memberless10          -0.3163      0.094     -3.347      0.001        -0.502    -0.131
Membermore10           0.6716      0.056     11.918      0.000         0.561     0.782
==============================================================================
Omnibus:                     2158.237   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           108203.895
Skew:                           3.456   Prob(JB):                         0.00
Kurtosis:                      33.459   Cond. No.                         67.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

REGION                  NaN
PONDERA                 1.0
HomeType                1.0
RoomsNumber             1.0
FloorMaterial           1.0
RoofMaterial            1.0
RoofCoat                1.0
Water                   1.0
WaterType               1.0
Toilet                  1.0
ToiletLocation          1.0
ToiletType              1.0
Sewer                   1.0
DumpSites               1.0
Flooding                1.0
EmergencyLoc            1.0
UsableTotalRooms        1.0
SleepingRooms           1.0
OfficeRooms             1.0
OnlyWork                1.0
Kitchen                 1.0
Sink                    1.0
Garage                  1.0
Ownership               1.0
CookingCombustible      1.0
BathroomUse             1.0
Working                 1.0
HouseMembers            1.0
Memberless10            1.0
Membermore10            1.0
TotalHouseHoldIncome    1.0
dtype: float64